# Homework 1 Review - Neural networks
# === CCM LAB EDITION ===



<img src="semcog_net.jpeg" style="width: 450px;"/>


## In this lab, we'll go over a simple variant of the network in Part C of the homework
* We'll start by loading in the data just like we would in the homework
* Then build a network for a **different** model architecture (see below)
* And demonstrate how you would train that model using the PyTorch functions

In [31]:
# Import libraries
from __future__ import print_function
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.nn.functional import sigmoid, relu
from scipy.cluster.hierarchy import dendrogram, linkage

Let's first load in the names of all the items, attributes, and relations into Python lists.

In [32]:
with open('data/sem_items.txt','r') as fid:
    names_items = np.array([l.strip() for l in fid.readlines()])
with open('data/sem_relations.txt','r') as fid:
    names_relations = np.array([l.strip() for l in fid.readlines()])
with open('data/sem_attributes.txt','r') as fid:
    names_attributes = np.array([l.strip() for l in fid.readlines()])
        
# number of objects / relations / attributes
nobj = len(names_items)
nrel = len(names_relations)
nattributes = len(names_attributes)

print('List of items:')
print(names_items)
print(nobj)

print("List of relations:")
print(names_relations)
print(nrel)

print("List of attributes:")
print(names_attributes)
print(nattributes)

List of items:
['Pine' 'Oak' 'Rose' 'Daisy' 'Robin' 'Canary' 'Sunfish' 'Salmon']
8
List of relations:
['ISA' 'Is' 'Can' 'Has']
4
List of attributes:
['Living thing' 'Plant' 'Animal' 'Tree' 'Flower' 'Bird' 'Fish' 'Pine'
 'Oak' 'Rose' 'Daisy' 'Robin' 'Canary' 'Sunfish' 'Salmon' 'Pretty' 'Big'
 'Living' 'Green' 'Red' 'Yellow' 'Grow' 'Move' 'Swim' 'Fly' 'Sing' 'Skin'
 'Roots' 'Leaves' 'Bark' 'Branch' 'Petals' 'Wings' 'Feathers' 'Gills'
 'Scales']
36


Next, let's load in the data matrix from a text file too. The matrix `D` has a row for each training pattern. It is split into a matrix of input patterns `input_pats` (item and relation) and their corresponding output patterns `output_pats` (attributes). There are `N` patterns total in the set.

For each input pattern, the first 8 elements indicate which item is being presented, and the next 4 indicate which relation is being queried. Each element of the output pattern corresponds to a different attribute. All patterns use 1-hot encoding.

In [33]:
# Next, let's load in the data matrix from a text file too.
D = np.loadtxt('data/sem_data.txt')
# The matrix D has a row for each training pattern.

# It is split into a matrix of input patterns input_pats (item and relation) 
input_pats = D[:,:nobj+nrel]
input_pats = torch.tensor(input_pats,dtype=torch.float)

# and their corresponding output patterns output_pats (attributes).
output_pats = D[:,nobj+nrel:]
output_pats = torch.tensor(output_pats,dtype=torch.float)

# The are N patterns total in the set.
N = input_pats.shape[0] # number of training patterns

input_v = input_pats[0,:].numpy().astype('bool')
output_v = output_pats[0,:].numpy().astype('bool')

print('Example input pattern:')
print(input_v.astype('int'))

print('Example output pattern:')
print(output_v.astype('int'))

print("")

print("Which encodes...")
print('Item ',end='')
print(names_items[input_v[:8]])

print('Relation ',end='')
print(names_relations[input_v[8:]])

print('Attributes ',end='')
print(names_attributes[output_v])

Example input pattern:
[1 0 0 0 0 0 0 0 1 0 0 0]
Example output pattern:
[1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Which encodes...
Item ['Pine']
Relation ['ISA']
Attributes ['Living thing' 'Plant' 'Tree' 'Pine']


# ========= CCM LAB EDITION ============


<img src="ugly.jpg" style="width: 350px;"/>



## Here, we will build an "ugly" version of the full model
* Each feature and relation is connected to a 15 node hidden layer and this layer is also connected to a 15 unit output layer.
* We will implement a relu function in the hidden layer and a sigmoid function in the output layer

In [43]:
class Net(nn.Module):
    def __init__(self, rep_size, hidden_size):
        super(Net, self).__init__()
        self.i2h = nn.Linear(8+4,15)
        self.h2o = nn.Linear(15,8+4)
        # add code

    def forward(self, x):
        x = x.view(-1,nobj+nrel) # reshape as size [B x (nobj+nrel) Tensor] if B=1
        hidden = self.i2h(x)
        hidden = relu(hidden)
        output = self.h2o(hidden)
        output = sigmoid(output)
        return output, hidden


We provide a completed function `train` for stochastic gradient descent. The network makes online (rather than batch) updates, adjusting its weights after the presentation of each input pattern.

In [44]:
def train(mynet,epoch_count,nepochs_additional=5000):
    #  mynet : Net class object
    #  epoch_count : (scalar) how many epochs have been completed so far
    #  nepochs_additional : (scalar) how many more epochs we want to run
    mynet.train()
    for e in range(nepochs_additional): # for each epoch
        error_epoch = 0.
        perm = np.random.permutation(N)
        for p in perm: # iterate through input patterns in random order
            mynet.zero_grad() # reset gradient
            output, hidden = mynet(input_pats[p,:]) # forward pass
            target = input_pats[p,:] 
            loss = criterion(output, target) # compute loss
            loss.backward() # compute gradient 
            optimizer.step() # update network parameters
            error_epoch += loss.item()
        error_epoch = error_epoch / float(N)        
        if e % 50 == 0:
            print('epoch ' + str(epoch_count+e) + ' loss ' + str(round(error_epoch,3)))
    return epoch_count + nepochs_additional

In [45]:
learning_rate = 0.1
criterion = nn.MSELoss() # mean squared error loss function
mynet = Net(rep_size=8,hidden_size=15)
optimizer = torch.optim.SGD(mynet.parameters(), lr=learning_rate) # stochastic gradient descent

epoch_count = 0
epoch_count = train(mynet,epoch_count,nepochs_additional=1000)


/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch 0 loss 0.25
epoch 50 loss 0.11
epoch 100 loss 0.074
epoch 150 loss 0.056
epoch 200 loss 0.037
epoch 250 loss 0.018
epoch 300 loss 0.009
epoch 350 loss 0.005
epoch 400 loss 0.003
epoch 450 loss 0.002
epoch 500 loss 0.002
epoch 550 loss 0.001
epoch 600 loss 0.001
epoch 650 loss 0.001
epoch 700 loss 0.001
epoch 750 loss 0.001
epoch 800 loss 0.001
epoch 850 loss 0.001
epoch 900 loss 0.001
epoch 950 loss 0.0


In [46]:
input_v = input_pats[0,:].numpy().astype('bool')

print('Example input pattern:')
print(input_v.astype('int'))

print('Item ',end='')
print(names_items[input_v[:8]])

print('Relation ',end='')
print(names_relations[input_v[8:]])

Example input pattern:
[1 0 0 0 0 0 0 0 1 0 0 0]
Item ['Pine']
Relation ['ISA']


In [47]:
output, hidden = mynet(input_pats[0,:]) # forward pass

/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [48]:
output

tensor([[0.9607, 0.0010, 0.0155, 0.0242, 0.0108, 0.0072, 0.0050, 0.0239, 0.9855,
         0.0108, 0.0140, 0.0028]], grad_fn=<SigmoidBackward>)

In [49]:
output.round()

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]],
       grad_fn=<RoundBackward>)

## If time:
* Concatenation

In [41]:
x = torch.rand(1, 2)
print(x)
y = torch.rand(1,2)
print(y)

tensor([[0.9075, 0.5936]])
tensor([[0.1407, 0.3876]])


In [42]:
concatenated = torch.cat((x,y),1)
print(concatenated)

tensor([[0.9075, 0.5936, 0.1407, 0.3876]])
